# Advent of Code: Day 6

### --- Day 6: Lanternfish ---
The sea floor is getting steeper. Maybe the sleigh keys got carried this way?

A massive school of glowing lanternfish swims past. They must spawn quickly to reach such large numbers - maybe exponentially quickly? You should model their growth rate to be sure.

Although you know nothing about this specific species of lanternfish, you make some guesses about their attributes. Surely, each lanternfish creates a new lanternfish once every 7 days.

However, this process isn't necessarily synchronized between every lanternfish - one lanternfish might have 2 days left until it creates another lanternfish, while another might have 4. So, you can model each fish as a single number that represents the number of days until it creates a new lanternfish.

Furthermore, you reason, a new lanternfish would surely need slightly longer before it's capable of producing more lanternfish: two more days for its first cycle.

So, suppose you have a lanternfish with an internal timer value of 3:

After one day, its internal timer would become 2.
After another day, its internal timer would become 1.
After another day, its internal timer would become 0.
After another day, its internal timer would reset to 6, and it would create a new lanternfish with an internal timer of 8.
After another day, the first lanternfish would have an internal timer of 5, and the second lanternfish would have an internal timer of 7.
A lanternfish that creates a new fish resets its timer to 6, not 7 (because 0 is included as a valid timer value). The new lanternfish starts with an internal timer of 8 and does not start counting down until the next day.

Realizing what you're trying to do, the submarine automatically produces a list of the ages of several hundred nearby lanternfish (your puzzle input). For example, suppose you were given the following list:

3,4,3,1,2
This list means that the first fish has an internal timer of 3, the second fish has an internal timer of 4, and so on until the fifth fish, which has an internal timer of 2. Simulating these fish over several days would proceed as follows:

Initial state: 3,4,3,1,2
After  1 day:  2,3,2,0,1
After  2 days: 1,2,1,6,0,8
After  3 days: 0,1,0,5,6,7,8
After  4 days: 6,0,6,4,5,6,7,8,8
After  5 days: 5,6,5,3,4,5,6,7,7,8
After  6 days: 4,5,4,2,3,4,5,6,6,7
After  7 days: 3,4,3,1,2,3,4,5,5,6
After  8 days: 2,3,2,0,1,2,3,4,4,5
After  9 days: 1,2,1,6,0,1,2,3,3,4,8
After 10 days: 0,1,0,5,6,0,1,2,2,3,7,8
After 11 days: 6,0,6,4,5,6,0,1,1,2,6,7,8,8,8
After 12 days: 5,6,5,3,4,5,6,0,0,1,5,6,7,7,7,8,8
After 13 days: 4,5,4,2,3,4,5,6,6,0,4,5,6,6,6,7,7,8,8
After 14 days: 3,4,3,1,2,3,4,5,5,6,3,4,5,5,5,6,6,7,7,8
After 15 days: 2,3,2,0,1,2,3,4,4,5,2,3,4,4,4,5,5,6,6,7
After 16 days: 1,2,1,6,0,1,2,3,3,4,1,2,3,3,3,4,4,5,5,6,8
After 17 days: 0,1,0,5,6,0,1,2,2,3,0,1,2,2,2,3,3,4,4,5,7,8
After 18 days: 6,0,6,4,5,6,0,1,1,2,6,0,1,1,1,2,2,3,3,4,6,7,8,8,8,8
Each day, a 0 becomes a 6 and adds a new 8 to the end of the list, while each other number decreases by 1 if it was present at the start of the day.

In this example, after 18 days, there are a total of 26 fish. After 80 days, there would be a total of 5934.

Find a way to simulate lanternfish. How many lanternfish would there be after 80 days?


### --- Part Two ---
Suppose the lanternfish live forever and have unlimited food and space. Would they take over the entire ocean?

After 256 days in the example above, there would be a total of 26984457539 lanternfish!

How many lanternfish would there be after 256 days?

In [86]:
lanternfish_sample = [3,4,3,1,2]

def lanternfish_input(input_file:str) -> list:
    data = []
    with open(input_file, 'r') as file:
        for line in file:
            try:
                for x in line.split(','):
                    data.append(int(x.strip()))
            except ValueError as e: 
                    pass
    return data

def age_lanternfish(lanternfish:list) -> list:
    new_fish_needed = 0
    for i, fish in enumerate(lanternfish):
        if fish > 0:
            lanternfish[i] -= 1
        else:
            lanternfish[i] = 6
            new_fish_needed += 1
    for new in range(new_fish_needed):
        lanternfish.append(8)
    return lanternfish

        

def age_x_days(lanternfish:list, days:int) -> list:
    for x in range(days):
        lanternfish = age_lanternfish(lanternfish)
    return lanternfish

In [87]:
# Workflow - Pt 1

# NOTE: this is a terrible method.
# method uses a single list item to represent one fish, 
# it's a good example of how exponential growth causes massive memory limitations
# and how NOT to store data.

lanternfish = lanternfish_input('data/d06_lanternfish.txt')

final_fish = age_x_days(lanternfish, 80)

print(f'The final number of lanternfish = {len(final_fish)}')

The final number of lanternfish = 352872


In [88]:
# Switching to method with a list indexes representing the ages and items being the counts

def lanternfish_dict(lanternfish:list, max_age:int) -> dict:
    age_dict = {}
    for i in range(max_age):
        age_dict[i] = 0
    for x in lanternfish:
        age_dict[x] += 1
    return age_dict

def lanternfish_ages(lanternfish:list, max_age:int) -> list:
    age_count = [0] * (max_age + 1)
    for fish in lanternfish:
        age_count[fish] += 1
    return age_count

def age_lanternfish_counts(lanternfish_counts:list, reprod_age:int, new_age:int) -> list:
    reprod_fish = lanternfish_counts[0]
    # Decrease index by 1 (reduce time to reproduce)
    lanternfish_counts = lanternfish_counts[1:]
    # Add new fish and reset reproducing fish
    lanternfish_counts.append(reprod_fish)
    lanternfish_counts[reprod_age] += reprod_fish
    return lanternfish_counts

def age_counts_x_days(lanternfish_counts:list, reprod_age:int, new_age:int, days:int) -> list:
    for day in range(days):
        lanternfish_counts = age_lanternfish_counts(lanternfish_counts, reprod_age, new_age)
    return lanternfish_counts

def sum_list(lanternfish_counts:list) -> int:
    count = 0
    for age in lanternfish_counts:
        count += age
    return count

In [89]:
# Workflow - Pt 2

# Changed to list of counts per age. List never exceeds max_age items.

lanternfish = lanternfish_input('data/d06_lanternfish.txt')

# Initial lanternfish age counts
lanternfish_counts = lanternfish_ages(lanternfish, 8)
# After 256 days
final_fish_counts = age_counts_x_days(lanternfish_counts, 6, 8, 256)
# Final number
num_fish = sum_list(final_fish_counts)

print(f'The final number of lanternfish = {num_fish}')

The final number of lanternfish = 1604361182149
